# Data Collection

Let's scrape data from the Good News Network website for, well, good news, and from huffpost for both good and bad news. This should give us a reasonably sized dataset of labeled examples!

In [1]:
import pandas as pd

# goodnews network and huffpost are two helper modules I've created to make scraping the sites much easier
import goodnewsnetwork
import huffpost

Setting max results to 10,000 for both good and bad news from HuffPost should hopefully catch all the articles. It seems unlikely that huffpost has written and categorized that many articles for both good and bad news, especially considering that they were launched in 2005. If they have, power to them, but I'm not sure we'll even *need* that much data!

In [ ]:
huff_good = huffpost.scrape_articles(category = 'good', max_results = 10000)
huff_bad = huffpost.scrape_articles(category = 'bad', max_results = 10000)

gnn_articles = goodnewsnetwork.scrape_pages()

huff_articles = huff_good + huff_bad + gnn_articles

articles_dataframe = pd.DataFrame(huff_articles)

Scraping Good News Network page 0/1019
Scraping Good News Network page 50/1019
Scraping Good News Network page 100/1019
Scraping Good News Network page 150/1019
Scraping Good News Network page 200/1019
Scraping Good News Network page 250/1019
Scraping Good News Network page 300/1019
Scraping Good News Network page 350/1019
Scraping Good News Network page 400/1019
Scraping Good News Network page 450/1019
Could not find description for article at URL: https://www.goodnewsnetwork.org/indiana-student-overcomes-stutter-deliver-inspiring-graduation-speech/
Scraping Good News Network page 500/1019
